In [4]:
import numpy as np
import matplotlib.pyplot as plt

Simulation avec le modèle de Heston pour gérer les trajectoires du portefeuille

In [5]:
def heston_simulation(S0, v0, mu, kappa, theta, sigma, rho, T, dt, n_paths):
    """
    Simule le modèle de Heston.
    Args:
        S0: Prix initial de l'actif
        v0: Variance initiale.
        mu: Taux de drift (rendement attendu).
        kappa: Taux de réversion
        theta: Niveau de variance de long terme.
        sigma: Vol de vol (volatilité de la variance).
        rho: Corrélation entre les deux mouvements browniens.
        T: Horizon temporel (en années).
        dt: Pas de temps.
        n_paths: Nombre de trajectoires simulées.
    Returns:
        S: Trajectoires de prix de l'actif (array).
        V: Trajectoires de variance (array).
        t: Temps simulés.
    """

    n_steps = int(T/ dt)
    t = np.linspace(0, T, n_steps, dtype=np.float32)

    #initialisation des matrices pour les prix et la variance
    S = np.zeros((n_paths, n_steps), dtype=np.float32)
    V = np.zeros((n_paths, n_steps), dtype=np.float32)

    #Conditions initiales
    S[:, 0] = S0
    V[:, 0] = v0

    #simuler les mouvements browniens corrélés
    Z1 = np.random.normal(size=(n_paths, n_steps - 1)).astype(np.float32)
    Z2 = np.random.normal(size=(n_paths, n_steps - 1)).astype(np.float32)
    W1 = Z1 * np.sqrt(dt).astype(np.float32)
    W2 = (rho * Z1 + np.sqrt(1 - rho**2) * Z2) * np.sqrt(dt)

    #Pré-calcul de certaines constantes
    sqrt_dt = np.sqrt(dt).astype(np.float32)
    max_variance_epsilon = 1e-6 #évite les valeurs négatives

    # Boucle temporelle pour simuler les trajectoires
    for i in range(1, n_steps):
        #Simulation de la variance v_t
        V[:, i] = V[:, i-1] + kappa * (theta - V[:, i-1]) * dt + sigma * np.sqrt(np.maximum(V[:, i-1], 0)) * np.sqrt(dt) * W2[:, i-1]
        V[:, i] = np.maximum(V[:, i], max_variance_epsilon) #Empèche les valeurs négatives

        #simulation du prix S_t
        S[:, i] = S[:, i-1] * np.exp((mu - 0.5 * V[:, i-1]) * dt + np.sqrt(V[:, i-1]) * W1[:, i-1])


    return S, V, t


Paramètres du modèle

In [6]:
S0 = 100 #Prix initial
v0 = 0.04 #Variance initiale
mu = 0.05 #Drift (rendement attendu)
kappa = 2.0 #Taux de réversion
theta = 0.04 #Variance long-terme
sigma = 0.5 #Vol de vol
rho = -0.7 #Corrélation
T = 1.0 #Horizon temporel (1 an)
dt = 0.01 #Pas de temps
n_paths = 500000 #Nombre de trajectoires

Simulation

In [7]:
S, V, t = heston_simulation(S0, v0, mu, kappa, theta, sigma, rho, T, dt, n_paths)

Visualisation des trajectoires

In [ ]:
plt.figure(figsize=(12,6))

#trajectoires de prix
plt.subplot(1, 2, 1)
for i in range(n_paths):
    plt.plot(t, S[i, :])
plt.title("Trajectoires de prix (Modèle de Heston)")
plt.xlabel("Temps (années)")
plt.ylabel("Prix de l'actif")
plt.legend()

#Trajectoires de variance
plt.subplot(1, 2, 2)
for i in range(n_paths):
    plt.plot(t, V[i, :])
plt.title("Trajectoire de variance (Modèle de Heston)")
plt.xlabel("Temps (années)")
plt.ylabel("Variance")
plt.legend()

plt.tight_layout()
plt.show()

/var/folders/3d/px25gn9n1cg8clmw10nxnsg00000gn/T/ipykernel_28670/382375617.py:10: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x106bcf380>>
Traceback (most recent call last):
  File "/Users/arthur/miniconda3/envs/Quadcopter_simulator/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
